In [ ]:
#default_exp helpers

In [ ]:
#hide
import pickle, os

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
REGION = 'ap-southeast-1'

In [ ]:
from villaProductDatabase.database import ProductDatabase
from villaProductDatabase.query import Querier

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from nicHelper.dictUtil import stripDict
from linesdk.linesdk import Line
from typing import List
import pandas as pd
import os, logging



# Helpers
## Static functions

In [ ]:
#export
class Helpers:
  pass

In [ ]:
class Tester(ProductDatabase, Helpers):
  class Meta:
    table_name = os.environ.get('DATABASE_TABLE_NAME')
    region = os.environ.get('REGION')
  pass

### send notification

In [ ]:
#export
@add_static_method(Helpers)
def notify(message):
  line = Line(os.environ.get('LINEKEY'))
  r = line.sendNotify(message)
  if r.status_code != 200:
    print(r.json())
    return False
  return True

In [ ]:
Tester.notify('hello')

True

## Class functions

### Get keys

In [ ]:
#export
@add_class_method(Helpers)
def keys(cls):
  keys = list(vars(cls)['_attributes'].keys())
  return keys

In [ ]:
Tester.keys()

['cprcode',
 'data',
 'iprcode',
 'lastUpdate',
 'needsUpdate',
 'oprcode',
 'pr_barcode',
 'pr_barcode2',
 'pr_dpcode',
 'pr_sa_method',
 'pr_sucode1',
 'pr_suref3',
 'sellingPrice']

### scan

In [ ]:
#export
@add_class_method(Helpers)
def scanDb(cls, limit=100):
  return list(cls.scan(limit=limit))

In [ ]:
%%time
items = Tester.scanDb(limit=1000)
len(items)

CPU times: user 173 ms, sys: 27.4 ms, total: 200 ms
Wall time: 360 ms


1000

## convert to df

In [ ]:
#export
@add_class_method(Helpers)
def toDf(cls, products:List[Helpers])->pd.DataFrame:
  df = pd.DataFrame()
  for product in products:
    productSeries = product.toSeries()
    productSeries = productSeries.rename(product.cprcode)
    df = df.append(productSeries)
  return df

In [ ]:
%%time
df = Tester.toDf(Tester.scan(limit=100))
df.head()

CPU times: user 1.27 s, sys: 102 µs, total: 1.27 s
Wall time: 1.28 s


,content_en,content_th,cprcode,hema_brand_en,hema_brand_th,hema_name_en,hema_name_th,hema_sizedesc,iprcode,online_category_l1_en,...,pr_suref3,prtype,psqty,pstype,sort_weight,villa_category_l1_en,villa_category_l2_en,villa_category_l3_en,villa_category_l4_en,0171670
0140942,0140942 Fragata Olive Pomace Oil 2L cooking oi...,Fragata Olive Pomace Oil 2L,0140942,Fragata,Fragata,Olive Pomace Oil,Olive Pomace Oil,2L,0140942,Grocery,...,A,I,1,1,0,Dry Grocery,Grocery,Oil & Seasoning & Sauce,Cooking Oil,NaN
0223750,0223750 Samyang Hot Chickhen Carbonarag 130g s...,Samyang Hot Chickhen Carbonarag 130g,0223750,Samyang,Samyang,Hot Chickhen Carbonarag,Hot Chickhen Carbonarag,130g,0223750,Grocery,...,A,I,1,1,0,Dry Grocery,Grocery,Rice & Noodles & Japanese,Noodles,NaN
0203790,0203790 KRISTY HAMPER 2016 SET G,KRISTY HAMPER 2016 SET G,0203790,,,KRISTY HAMPER 2016 SET G,KRISTY HAMPER 2016 SET G,,0203790,,...,A,I,1,1,0,Dry Grocery,Grocery,Oil & Seasoning & Sauce,Baking Needs,NaN
0207006,0207006 Malee 100% Tangerine Sukhothai 1L juice,Malee 100% Tangerine Sukhothai 1L น้ำผลไม้,0207006,Malee,Malee,100% Tangerine Sukhothai,100% Tangerine Sukhothai,1L,0207006,Drinks,...,C,I,1,1,0,Dry Grocery,Alcohol & Beverage,Juice,Juice Local,NaN
235078,0235078 PLANEAT ROASTED SEASONED AJI BLACK PEPPE,PLANEAT ROASTED SEASONED AJI BLACK PEPPE,235078,none,none,PLANEAT ROASTED SEASONED AJI BLACK PEPPE,PLANEAT ROASTED SEASONED AJI BLACK PEPPE,none,235078,none,...,A,I,1.0,1.0,0.0,none,none,none,none,NaN


### From Series

In [ ]:
#export
@add_class_method(Helpers)
def fromSeries(cls, data:pd.Series)->Helpers:
  return cls.fromDict(data.to_dict())
  

In [ ]:
item = df.iloc[0]
type(item)
result = Tester.fromSeries(item)
result.data

{'content_en': '0140942 Fragata Olive Pomace Oil 2L cooking oil, oil cooking,Fragata Olive Pomace Oil, Fragata, Olive purity pomace oil, Omace oil olive pomace oil, cooking olive pomace oil, olive pomace oil cooking',
 'content_th': 'Fragata Olive Pomace Oil 2L',
 'cprcode': '0140942',
 'hema_brand_en': 'Fragata',
 'hema_brand_th': 'Fragata',
 'hema_name_en': 'Olive Pomace Oil',
 'hema_name_th': 'Olive Pomace Oil',
 'hema_sizedesc': '2L',
 'iprcode': '0140942',
 'online_category_l1_en': 'Grocery',
 'online_category_l1_th': 'สินค้าบริโภค',
 'online_category_l2_en': 'Oils & Vinegar',
 'online_category_l2_th': 'น้ำมันและน้ำส้มสายชู',
 'online_category_l3_en': '',
 'online_category_l3_th': '',
 'oprcode': '0140942',
 'ordertype': 'Y',
 'pr_abb': 'FRAGATA OLIVE POMACE',
 'pr_active': 'Y',
 'pr_barcode': '8410134118601',
 'pr_barcode2': '841013411860',
 'pr_brand_en': 'Fragata',
 'pr_brand_th': 'Fragata',
 'pr_cgcode': '01',
 'pr_code': '0140942',
 'pr_country_en': '',
 'pr_country_th': '',


### From DF

In [ ]:
#export
@add_class_method(Helpers)
def fromDf(cls, data:pd.DataFrame)->List[Helpers]:
  return data.apply(lambda x: cls.fromSeries(x), axis=1).to_list()
  

In [ ]:
Tester.fromDf(df)[0]

### From dict

In [ ]:
#export
@add_class_method(Helpers)
def fromDict(cls, dictInput):
  logging.debug(dictInput)
  dictInput = stripDict(dictInput)
  #### extract the keys
  filteredInput = {k:v for k,v in dictInput.items() if k in cls.keys()}
  #### save whole object to dictInput
  filteredInput['data'] = dictInput
  logging.debug(filteredInput)
  return cls(**filteredInput)

In [ ]:
item = {
  'cprcode': '0171670',
  'iprcode': '0171670',
  'oprcode': '0171670',
  'ordertype': 'Y',
  'pr_abb': 'JIRAPAT YOUNG KALE 2',
  'pr_active': 'Y',
  'pr_cgcode': '05',
  'pr_code': '0171670',
  'pr_dpcode': '19',
  'pr_engname': 'JIRAAT YOUNG KALE 200 G.',
  'pr_ggcode': '057',
  'pr_market': 'JIRAPAT ยอดคะน้า 200 G.',
  'pr_name': 'JIRAPAT ยอดคะน้า 200 G.',
  'pr_puqty': '1',
  'pr_sa_method': '1',
  'pr_sucode1': 'CM845',
  'pr_suref3': 'A',
  'prtype': 'I',
  'psqty': '1',
  'pstype': '1'}
Tester.fromDict(item)

### toListOfDict

In [ ]:
#export
@add_class_method(Helpers)
def toListDict(cls, items:List[Helpers]):
  return [item.data for item in items]

In [ ]:
items = Tester.fromDf(df)
len(Tester.toListDict(items))

## Standard functions

### toSeries 

In [ ]:
#export
@add_method(Helpers)
def toSeries(self):
  return pd.Series(self.data)

In [ ]:
item = next(Tester.query('0000009'))
data = item.data
item.toSeries()

### Set update status

In [ ]:
#export
@add_method(Helpers)
def setNoUpdate(self, batch = None):
  self.needsUpdate = self.FALSE
  if batch:
    return batch.save(self)
  else:
    return self.save()
  
@add_method(Helpers)
def setUpdate(self, batch=None):
  self.needsUpdate = self.TRUE
  if batch:
    return batch.save(self)
  else:
    return self.save()

In [ ]:
%%time
sku = '0000009'
item = next(ProductDatabase.query(sku))
item.setUpdate()
item = next(ProductDatabase.query(sku))
print(item.needsUpdate)
item.setNoUpdate()
item = next(ProductDatabase.query(sku))
print(item.needsUpdate)